In [1]:
import os
import warnings
import pathlib

import earthpy as et
import geopandas as gpd
import geoviews as gv
import holoviews as hv
import hvplot.pandas
import pandas as pd
import pyogrio

# Ignore FutureWarning coming from hvplot
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
states_url = 'http://eric.clst.org/assets/wiki/uploads/Stuff/gz_2010_us_040_00_5m.json'
states_gdf = gpd.read_file(states_url)

In [3]:
# states_gdf.hvplot(geo=True)

In [4]:
states_gdf_utm = states_gdf.to_crs('EPSG:5498')

In [5]:
# states_gdf_utm.hvplot(aspect='equal', title='UTM 12N') * hv.VLine(500000).opts(color='r', line_width=0.5)

In [6]:
# Data for fire occurrence

fire_url = (
    "https://www.fs.usda.gov/rds/archive/products/RDS-2013-0009.6"
    "/RDS-2013-0009.6_Data_Format2_GDB.zip")
fire_dir = et.data.get_data(url = fire_url)

In [7]:
# Call dataframe

fire_path = os.path.join(fire_dir, 'Data', 'FPA_FOD_20221014.gdb')
if not 'fire_gdf' in globals():
    fire_gdf = pyogrio.read_dataframe(fire_path, layer='Fires')

In [8]:
# Clean data

fire_clean_gdf = (
    fire_gdf
    [['FOD_ID', 'DISCOVERY_DATE', 'FIRE_SIZE', 'geometry']]
      .set_index('FOD_ID')
)
fire_clean_gdf.DISCOVERY_DATE = pd.to_datetime(fire_clean_gdf.DISCOVERY_DATE)
fire_clean_gdf = fire_clean_gdf.to_crs(states_gdf_utm.crs)

In [9]:
fire_region_gdf = (
    states_gdf_utm
    [['NAME', 'geometry']]
    .sjoin(fire_clean_gdf, how='inner', predicate='intersects')
)
fire_region_gdf = (
    fire_region_gdf
    #[['name', 'DISCOVERY_DATE', 'FIRE_SIZE']]
    .groupby(['NAME', fire_region_gdf.DISCOVERY_DATE.dt.year])
    .agg(
        max_fire_size=('FIRE_SIZE', 'max'), 
        num_fires=('index_right', 'count'))
)

In [10]:
# Compute area of regional watersheds

states_gdf_utm['area_ha'] = (
    states_gdf_utm.to_crs(9822).area
    # Convert to hectares
    / 10000
    # Convert to millions of hectares
    / 1000000
)

# Computing total fires in each watershed
fire_count_df = (
    fire_region_gdf
    .reset_index()
    [['NAME', 'num_fires']]
    .groupby('NAME')
    .sum())

# Add the area and geometry back
fire_density_gdf = (
    states_gdf_utm
    .set_index('NAME')
    .join(fire_count_df)
    [['num_fires', 'area_ha', 'geometry']])

# Computing fire density
fire_density_gdf['fire_density_per_ha'] = (
    fire_density_gdf.num_fires / fire_density_gdf.area_ha)

# fire_density_gdf.fire_density_per_ha

# 30 Years of Wildfire Data Displays Which States are Most Susceptible to Climate Change

In [11]:
# Set values for ylabels and titles
labels = pd.DataFrame(dict(
    column_name = ['max_fire_size', 'num_fires'],
    ylabel = ['Fire Size (million ha)', 'Number of Fires'],
    title = ['Largest fire on record in the region',
             'Total number of fires in the region']))

def fire_plot(region_name, df=fire_region_gdf, labels=labels):
    """
    Create a multi-panel plot for a region

    Parameters
    ----------
    region_name : str
      The name of the region to generate a plot for. Must exists 
      in the 'name' index of df.
    df : pd.DataFrame
      The dataframe with the data to plot. Columns much match
      an item in labels.column_name to be plotted
    labels : pd.DataFrame
      Plot labels. Must have a 'column_name', 'ylabel', and 'title'
      columns with str values. Each row will be a subplot.

    Returns
    -------
    plot : hv.core.layout.Layout
      A holoviews plot layout or similar. For use with hv.DynamicMap.
    """
    # Generate a subplot for each row in the labels
    subplots = []
    # Iterate through the labels row by row
    for i, labs in labels.iterrows():
        # Create subplot
        subplot = (
            df.xs(region_name, level='NAME')
            [[labs.column_name]]
            .hvplot(
                xlabel='Year', ylabel=labs.ylabel,
                title=labs.title, width=1000,
                color='red', size=2))
        # Year should be x value, column name should be y value
        subplots.append(subplot)

    # Stack the subplots vertically
    plot = hv.Layout(subplots).cols(1)
    return plot

# Create a dropdown menu to switch between regions
(
    hv.DynamicMap(
        # The plotting function for the two-panel fire history
        fire_plot,
        # Define the dimension for the dropdown
        kdims=[('region', 'Region')])
    # Add the explicit indexing - region names as a bokeh dimension
    .redim.values(region=fire_region_gdf.reset_index().NAME)
)

/opt/conda/lib/python3.10/site-packages/holoviews/plotting/bokeh/plot.py:987: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  layout_plot = gridplot(
/opt/conda/lib/python3.10/site-packages/holoviews/plotting/bokeh/plot.py:987: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  layout_plot = gridplot(


BokehModel(combine_events=True, render_bundle={'docs_json': {'b9e9447c-bee4-4a26-8410-fe03688c690e': {'version…

Frequency and size of wildfires continues to change in the United States with climate change. It is predicted that about half of the amount of burned acreage would have been burned without the existence of climate change (Pierre-louis & Popovich 2018). In addition, less snowpack in the winters has led to a smaller amount of water available in the summer. With all of these factors put together, wildfires continue to increase.

The data used in this analysis is a spatial database of wildfires in the US from 1992 to 2020. In the data, there is discovery date, final fire size, and point location. 

Below is a map showing the number of fires per million acres in each state in the continental United States.

In [12]:
fire_density_gdf.geometry = fire_density_gdf.geometry.simplify(tolerance=.1)

In [13]:
(
    fire_density_gdf
    .reset_index()
    [['fire_density_per_ha', 'NAME', 'geometry']],
    #vdims=['fire_density_per_ha', 'name']
)

poly_plot = (
    gv.Polygons(
        fire_density_gdf
            .drop(['Alaska', 'Hawaii'], axis='rows')
            .reset_index()
            .dropna()
            [['fire_density_per_ha', 'geometry']])
        .opts(
            width=600, height=600,
            colorbar=True, color='fire_density',
            cmap='plasma', line_color='white',
            title="Density of Fires per Million Acres by State",
            xaxis='bare', yaxis='bare', tools=['hover'],
        )
)

(gv.tile_sources.OSM * poly_plot)

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]   (fire_density_per_ha)

### Citations

Pierre-louis, K., Popovich, N. (2018, November 27). Climate change is fueling wildfires nationwide, New Report warns. The New York Times. https://www.nytimes.com/interactive/2018/11/27/climate/wildfire-global-warming.html 

State boundaries retrieved from [SnowEx Hackweek](https://snowex-2021.hackweek.io/tutorials/geospatial/vector.html).

Fire data retrieved from [USDA](https://www.fs.usda.gov/rds/archive/Catalog/RDS-2013-0009.6).

In [14]:
%%capture
%%bash
jupyter nbconvert fireboundaries-vector.ipynb --to html --no-input